In [35]:
import os
import re
import json
import time
import pandas as pd
import numpy as np
from typing import List, Dict
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer

In [ ]:
# 입력/출력 경로 설정
MD_INPUT_DIR = r"c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output"
JSON_OUTPUT_DIR = r"c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\json폴더"
JSON_OUTPUT_FILE = os.path.join(JSON_OUTPUT_DIR, "법률_임베딩.json")

# 입력 마크다운 파일들 수집
md_files = sorted([f for f in os.listdir(MD_INPUT_DIR) if f.endswith('.md')])
print(f"발견된 마크다운 파일: {len(md_files)}개")
for fname in md_files:
    print(f"  - {fname}")

📁 발견된 마크다운 파일: 9개
  - 112신고.md
  - 가사근로자.md
  - 간선급버스운행.md
  - 간호법.md
  - 감사원징계.md
  - 개인정보호법.md
  - 법원설치.md
  - 소비세법.md
  - 온실가스배출.md


In [ ]:
# 임베딩 모델 로드
print("🧠 임베딩 모델 로드 중...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
print("✅ 임베딩 모델 로드 완료")

# 출력 폴더 생성
os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

print(f"입력 디렉토리: {MD_INPUT_DIR}")
print(f"마크다운 파일 수: {len(md_files)}개")
print(f"출력 파일: {JSON_OUTPUT_FILE}")
print(f"출력 디렉토리 준비 완료")

🧠 임베딩 모델 로드 중...
✅ 임베딩 모델 로드 완료
📁 입력 디렉토리: c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output
📁 마크다운 파일 수: 9개
💾 출력 파일: c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\json폴더\법률_임베딩.json
✅ 출력 디렉토리 준비 완료


In [ ]:
def get_embedding_from_ollama(text: str) -> np.ndarray:
    """
    SentenceTransformer를 통해 텍스트의 임베딩 벡터 생성
    
    Args:
        text: 임베딩할 텍스트
        
    Returns:
        np.ndarray: 임베딩 벡터
    """
    # Nomic 모델 권장 접두사 추가 (검색 품질 향상)
    input_text = f"search_document: {text}"
    dense_vec = model.encode(input_text, normalize_embeddings=True)
    
    return dense_vec


def parse_markdown_hierarchical(md_content: str) -> List[Dict]:
    """
    마크다운 문서를 계층적 구조로 파싱
    H1(법률명) > H2(장) > H3(조) > H4(항) 구조 유지
    H5 메타정보(담당부서, 법령정보, 약칭, 메타정보)는 필터링
    """
    lines = md_content.split('\n')
    hierarchy = []
    
    current_h1 = None
    current_h2 = None
    current_h3 = None
    current_h4 = None
    current_h5 = None
    current_content = []
    skip_next_lines = False  # H5 메타정보 스킵 플래그
    
    for line in lines:
        # 헤더 레벨 감지
        h1_match = re.match(r'^#\s+(.+?)$', line)
        h2_match = re.match(r'^##\s+(.+?)$', line)
        h3_match = re.match(r'^###\s+(.+?)$', line)
        h4_match = re.match(r'^####\s+(.+?)$', line)
        h5_match = re.match(r'^#####\s+(.+?)$', line)
        
        # H5 메타정보 감지 (담당부서, 법령정보, 약칭, 메타정보)
        if h5_match:
            h5_text = h5_match.group(1).strip()
            if h5_text in ['담당부서', '법령정보', '약칭', '메타정보']:
                skip_next_lines = True
                continue
        
        # H5 메타정보 직후 본문은 스킵
        if skip_next_lines and line.strip() and not re.match(r'^#{1,5}\s+', line):
            continue
        elif skip_next_lines and (h1_match or h2_match or h3_match or h4_match):
            skip_next_lines = False
        
        if h1_match:
            current_h1 = h1_match.group(1).strip()
            current_h2 = None
            current_h3 = None
            current_h4 = None
            current_content = []
            skip_next_lines = False
        elif h2_match:
            current_h2 = h2_match.group(1).strip()
            current_h3 = None
            current_h4 = None
            current_content = []
            skip_next_lines = False
        elif h3_match:
            # 이전 H3의 내용 저장
            if current_h3 and current_content:
                hierarchy.append({
                    'level': 3,
                    'h1': current_h1,
                    'h2': current_h2,
                    'h3': current_h3,
                    'h4': current_h4,
                    'content': '\n'.join(current_content).strip()
                })
            current_h3 = h3_match.group(1).strip()
            current_h4 = None
            current_content = []
            skip_next_lines = False
        elif h4_match:
            # 이전 H4의 내용 저장
            if current_h4 and current_content:
                hierarchy.append({
                    'level': 4,
                    'h1': current_h1,
                    'h2': current_h2,
                    'h3': current_h3,
                    'h4': current_h4,
                    'content': '\n'.join(current_content).strip()
                })
            current_h4 = h4_match.group(1).strip()
            current_content = []
            skip_next_lines = False
        elif line.strip() and not skip_next_lines:  # 빈 줄이 아니고 H5 스킵 아닌 경우
            current_content.append(line)
    
    # 마지막 항목 저장
    if current_h4 and current_content:
        hierarchy.append({
            'level': 4,
            'h1': current_h1,
            'h2': current_h2,
            'h3': current_h3,
            'h4': current_h4,
            'content': '\n'.join(current_content).strip()
        })
    elif current_h3 and current_content:
        hierarchy.append({
            'level': 3,
            'h1': current_h1,
            'h2': current_h2,
            'h3': current_h3,
            'h4': None,
            'content': '\n'.join(current_content).strip()
        })
    
    return hierarchy


def create_hierarchical_chunks(hierarchy_list: List[Dict], max_chars: int = 1500) -> List[Dict]:
    """
    계층적 구조를 청크로 변환
    같은 조(H3) 내의 여러 항(H4)을 하나의 청크로 병합 가능
    """
    chunks = []
    temp_chunk = {
        'h1': None,
        'h2': None,
        'h3': None,
        'items': []  # H4 항목들
    }
    temp_size = 0
    
    for item in hierarchy_list:
        item_text = f"{item.get('h4', '')}\n{item['content']}"
        item_size = len(item_text)
        
        # 같은 조(H3)인 경우
        if temp_chunk['h3'] == item['h3'] and temp_size + item_size <= max_chars:
            temp_chunk['items'].append(item)
            temp_size += item_size
        else:
            # 새로운 청크 시작
            if temp_chunk['items']:
                chunks.append(temp_chunk.copy())
            temp_chunk = {
                'h1': item['h1'],
                'h2': item['h2'],
                'h3': item['h3'],
                'items': [item]
            }
            temp_size = item_size
    
    # 마지막 청크 추가
    if temp_chunk['items']:
        chunks.append(temp_chunk)
    
    return chunks


def chunk_to_text(chunk: Dict) -> str:
    """
    청크를 텍스트 형태로 변환 (임베딩용)
    """
    text_parts = []
    
    if chunk['h1']:
        text_parts.append(f"# {chunk['h1']}")
    if chunk['h2']:
        text_parts.append(f"## {chunk['h2']}")
    if chunk['h3']:
        text_parts.append(f"### {chunk['h3']}")
    
    for item in chunk['items']:
        if item['h4']:
            text_parts.append(f"#### {item['h4']}")
        if item['content']:
            text_parts.append(item['content'])
    
    return '\n\n'.join(text_parts)


print("함수 정의 완료")

✅ 함수 정의 완료


In [ ]:
# 모든 마크다운 파일 읽기 및 파싱
print("📖 마크다운 파일 읽기 및 파싱 시작...\n")

all_hierarchy = []
file_stats = {}

for md_file in md_files:
    md_path = os.path.join(MD_INPUT_DIR, md_file)
    print(f"📄 처리 중: {md_file}")
    
    try:
        with open(md_path, 'r', encoding='utf-8') as f:
            md_content = f.read()
        
        # 계층적 구조로 파싱
        hierarchy = parse_markdown_hierarchical(md_content)
        
        # 파일별 통계
        file_stats[md_file] = {
            'items': len(hierarchy),
            'size': len(md_content)
        }
        
        # 전역 계층 구조에 추가 (파일명 정보 포함)
        for item in hierarchy:
            item['source_file'] = md_file
            all_hierarchy.append(item)
        
        print(f"  ✓ {len(hierarchy)}개 항목 추출\n")
    except Exception as e:
        print(f"  ✗ 오류: {str(e)}\n")

print(f"파싱 완료: 총 {len(all_hierarchy)}개의 항목 추출")
print(f"\n파일별 통계:")
for fname, stats in file_stats.items():
    print(f"  - {fname}: {stats['items']}개 항목, {stats['size']/1024:.1f} KB")

📖 마크다운 파일 읽기 및 파싱 시작...

📄 처리 중: 112신고.md
  ✓ 40개 항목 추출

📄 처리 중: 가사근로자.md
  ✓ 16개 항목 추출

📄 처리 중: 간선급버스운행.md
  ✓ 50개 항목 추출

📄 처리 중: 간호법.md
  ✓ 117개 항목 추출

📄 처리 중: 감사원징계.md
  ✓ 66개 항목 추출

📄 처리 중: 개인정보호법.md
  ✓ 372개 항목 추출

📄 처리 중: 법원설치.md
  ✓ 4개 항목 추출

📄 처리 중: 소비세법.md
  ✓ 118개 항목 추출

📄 처리 중: 온실가스배출.md
  ✓ 37개 항목 추출

✅ 파싱 완료: 총 820개의 항목 추출

📊 파일별 통계:
  - 112신고.md: 40개 항목, 5.4 KB
  - 가사근로자.md: 16개 항목, 4.6 KB
  - 간선급버스운행.md: 50개 항목, 16.1 KB
  - 간호법.md: 117개 항목, 20.3 KB
  - 감사원징계.md: 66개 항목, 10.5 KB
  - 개인정보호법.md: 372개 항목, 75.4 KB
  - 법원설치.md: 4개 항목, 1.3 KB
  - 소비세법.md: 118개 항목, 34.7 KB
  - 온실가스배출.md: 37개 항목, 6.7 KB


In [ ]:
# 파일 단위 청킹
print("📦 파일 단위 청킹 수행 중...")

chunks = []
for md_file in md_files:
    # 해당 파일의 모든 계층 항목
    file_items = [item for item in all_hierarchy if item['source_file'] == md_file]
    
    if not file_items:
        continue
    
    # 파일 내 모든 항목을 하나의 청크로 병합
    chunk = {
        'source_file': md_file,
        'h1': file_items[0].get('h1'),  # 첫 H1 (법률명)
        'h2': None,
        'h3': None,
        'items': file_items  # 파일의 모든 항목
    }
    chunks.append(chunk)

print(f"✅ 파일 단위 청킹 완료: {len(chunks)}개 청크 생성")

# 청크 통계
chunk_sizes = [len(chunk_to_text(chunk)) for chunk in chunks]
print(f"\n 청크 크기 통계:")
print(f"   - 최소: {min(chunk_sizes)} 글자")
print(f"   - 최대: {max(chunk_sizes)} 글자")
print(f"   - 평균: {sum(chunk_sizes) / len(chunk_sizes):.0f} 글자")
print(f"\n 파일별 청크 크기:")
for i, (fname, size) in enumerate(zip(md_files, chunk_sizes), 1):
    print(f"   {i}. {fname}: {size} 글자")

📦 파일 단위 청킹 수행 중...
✅ 파일 단위 청킹 완료: 9개 청크 생성

📊 청크 크기 통계:
   - 최소: 347 글자
   - 최대: 68592 글자
   - 평균: 17281 글자

📋 파일별 청크 크기:
   1. 112신고.md: 4457 글자
   2. 가사근로자.md: 3502 글자
   3. 간선급버스운행.md: 11963 글자
   4. 간호법.md: 17524 글자
   5. 감사원징계.md: 9627 글자
   6. 개인정보호법.md: 68592 글자
   7. 법원설치.md: 347 글자
   8. 소비세법.md: 33046 글자
   9. 온실가스배출.md: 6468 글자


In [ ]:
import hashlib

print("임베딩 생성 중...")

# 파일별 JSON 결과 저장소
file_json_results = {}

for chunk_idx, chunk in enumerate(chunks):
    chunk_text = chunk_to_text(chunk)
    
    # 임베딩 생성
    dense_vec = get_embedding_from_ollama(chunk_text)
    
    source_file = chunk['source_file']
    
    # 파일 해시 생성
    file_hash = hashlib.md5(source_file.encode()).hexdigest()
    
    # JSON 항목 생성
    json_item = {
        "pk": f"LAW_{chunk_idx+1:03d}",
        "source": source_file,
        "text": chunk_text,
        "dense": dense_vec.tolist(),
        "file_hash": file_hash,
        "page": 1,
        "row": chunk_idx + 1
    }
    
    # 파일별로 분류
    if source_file not in file_json_results:
        file_json_results[source_file] = []
    
    file_json_results[source_file].append(json_item)
    
    print(f"   [{chunk_idx+1}/{len(chunks)}] {source_file}: {len(chunk['items'])} 항목, {len(chunk_text)} 글자")

print(f"\n 임베딩 완료: {len(chunks)}개의 청크")

# 편의상 전체 결과도 보관
json_results = [item for items in file_json_results.values() for item in items]

🧠 임베딩 생성 중...
   [1/9] 112신고.md: 40 항목, 4457 글자
   [2/9] 가사근로자.md: 16 항목, 3502 글자
   [3/9] 간선급버스운행.md: 50 항목, 11963 글자
   [4/9] 간호법.md: 117 항목, 17524 글자
   [5/9] 감사원징계.md: 66 항목, 9627 글자
   [6/9] 개인정보호법.md: 372 항목, 68592 글자
   [7/9] 법원설치.md: 4 항목, 347 글자
   [8/9] 소비세법.md: 118 항목, 33046 글자
   [9/9] 온실가스배출.md: 37 항목, 6468 글자

✅ 임베딩 완료: 9개의 청크


In [ ]:
# JSON 저장
print(f"JSON 파일 저장 중: {JSON_OUTPUT_FILE}")
with open(JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=2)

print(f"JSON 저장 완료")

# 파일 크기 확인
file_size_mb = os.path.getsize(JSON_OUTPUT_FILE) / (1024 * 1024)
print(f"파일 크기: {file_size_mb:.2f} MB")

💾 JSON 파일 저장 중: c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\json폴더\법률_임베딩.json
✅ JSON 저장 완료
📊 파일 크기: 0.53 MB


In [ ]:
# 결과 검증
print("\n 생성된 JSON 구조:")
print(f"\n총 청크 수: {len(json_results)}")
print(f"\n첫 번째 청크 샘플:")
sample_chunk = json_results[0]
print(f"  PK: {sample_chunk['pk']}")
print(f"  원본 파일: {sample_chunk['source_file']}")
print(f"  법률명: {sample_chunk['h1']}")
print(f"  포함 항목: {sample_chunk['item_count']}개")
print(f"  Text 길이: {len(sample_chunk['text'])} 글자")
print(f"  Embedding 차원: {len(sample_chunk['dense'])}")

# 통계
print(f"\n 최종 통계:")
print(f"  - 처리 파일: {len(md_files)}개")
print(f"  - 추출 항목: {len(all_hierarchy)}개")
print(f"  - 생성 청크: {len(json_results)}개 (파일 단위)")
print(f"  - 평균 청크 크기: {sum(chunk_sizes) / len(chunk_sizes):.0f} 글자")
print(f"  - 임베딩 차원: {len(sample_chunk['dense'])}")

print("\n 모든 프로세스 완료!")


📋 생성된 JSON 구조:

총 청크 수: 9

첫 번째 청크 샘플:
  PK: LAW_CHUNK_001
  원본 파일: 112신고.md
  법률명: None
  포함 항목: 40개
  Text 길이: 4457 글자
  Embedding 차원: 768

📊 최종 통계:
  - 처리 파일: 9개
  - 추출 항목: 820개
  - 생성 청크: 9개 (파일 단위)
  - 평균 청크 크기: 17281 글자
  - 임베딩 차원: 768

✅ 모든 프로세스 완료!


In [ ]:
# 쿼리 예시
queries = [
    "고용보험 가입",
    "시행일",
    "담당부서 연락처"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f" 쿼리: '{query}'")
    print(f"{'='*60}")
    
    query_embedding = model.encode([f"search_document: {query}"], normalize_embeddings=True)[0]
    
    # 코사인 유사도 계산
    similarities = []
    for chunk in json_results:
        chunk_embedding = np.array(chunk['dense'])
        similarity = np.dot(query_embedding, chunk_embedding)
        similarities.append((similarity, chunk))
    
    # 상위 3개 결과
    similarities.sort(reverse=True, key=lambda x: x[0])
    
    print("\n 상위 3개 관련 청크:\n")
    for i, (score, chunk) in enumerate(similarities[:3], 1):
        print(f"[{i}] 유사도: {score:.4f}")
        print(f"    파일: {chunk['source_file']}")
        print(f"    법률: {chunk['h1']}")
        print(f"    장: {chunk['h2']}")
        print(f"    조: {chunk['h3']}")
        print(f"    내용: {chunk['text'][:120]}...\n")


🔍 쿼리: '고용보험 가입'

📌 상위 3개 관련 청크:

[1] 유사도: 0.4918
    파일: 가사근로자.md
    법률: None
    장: None
    조: 제1조(목적) 이 영은 「가사근로자의 고용개선 등에 관한 법률」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을
    내용: ### 제1조(목적) 이 영은 「가사근로자의 고용개선 등에 관한 법률」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을

#### 제1항

제2조(가사서비스 제공기관의 인증 요건) 「가사근로자의 고용개선 등에 관...

[2] 유사도: 0.4620
    파일: 가사근로자.md
    법률: None
    장: None
    조: 제9조(권한의 위임) 고용노동부장관은 법 제24조제1항에 따라 다음 각 호의 사항에 관한 권한을 지방고용노동관서의
    내용: ### 제9조(권한의 위임) 고용노동부장관은 법 제24조제1항에 따라 다음 각 호의 사항에 관한 권한을 지방고용노동관서의

#### 제1항

제10조(업무의 위탁) 법 제24조제2항제3호에서 “대통령령으로 정하는 업...

[3] 유사도: 0.4561
    파일: 가사근로자.md
    법률: None
    장: None
    조: 제4조(근로계약 체결 시 명시해야 할 근로조건) 법 제14조제1항제5호에서 “대통령령으로 정하는 근로조건에 관한 사항
    내용: ### 제4조(근로계약 체결 시 명시해야 할 근로조건) 법 제14조제1항제5호에서 “대통령령으로 정하는 근로조건에 관한 사항

”이란 다음 각 호의 사항을 말한다.
가사근로자의 고용개선 등에 관한 법률 시행령
- 1...


🔍 쿼리: '시행일'

📌 상위 3개 관련 청크:

[1] 유사도: 0.5472
    파일: 간호법.md
    법률: None
    장: 부칙
    조: 제1조(시행일) 이 법은 공포 후 9개월이 경과한 날부터 시행한다. 다만, 제14조제2항에 따른 교육과정 운영기관의 지
    내용: ## 부칙

### 

In [ ]:
import os
import re
import json
import hashlib
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 환경 설정
MD_INPUT_DIR = r"./output" 
JSON_OUTPUT_DIR = r"./json_result"
os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

# 2. 모델로드 (사용자 모델 유지)
print(" 임베딩 모델 로드 중...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# 3. 텍스트 스플리터 설정 (1000자 제한 + 문맥 보존)
# 문단( \n\n ), 줄바꿈( \n ), 문장( . ), 공백 순으로 경계를 찾습니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,  # 문맥 연결을 위해 앞뒤 100자 정도 겹침 허용
    separators=["\n\n", "\n", ". ", " ", ""],
    length_function=len,
)

def process_md_to_json():
    md_files = [f for f in os.listdir(MD_INPUT_DIR) if f.endswith('.md')]
    
    for md_file in md_files:
        print(f"📄 처리 중: {md_file}")
        md_path = os.path.join(MD_INPUT_DIR, md_file)
        
        with open(md_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 파일 전체에 대한 해시 생성 (파일 식별용)
        file_hash = hashlib.md5(content.encode()).hexdigest()
        
        # 1000자 단위로 지능형 분할
        chunks = text_splitter.split_text(content)
        
        json_results = []
        for i, chunk_text in enumerate(chunks):
            # 임베딩 생성
            dense_vec = model.encode(f"search_document: {chunk_text}", normalize_embeddings=True)
            
            # PK 생성 (파일명_순번) - 동일 파일 내 순서 식별
            file_name_only = os.path.splitext(md_file)[0]
            pk = f"{file_name_only}_{i+1:03d}"
            
            # 데이터 구조화
            item = {
                "pk": pk,
                "source": md_file,          # 원본 파일 식별
                "text": chunk_text,
                "dense": dense_vec.tolist(),
                "file_hash": file_hash,     # 파일 버전/내용 식별
                "page": 1,                  # MD 특성상 1로 고정
                "row": i + 1                # 파일 내 순서
            }
            json_results.append(item)
        
        # 파일별 개별 JSON 저장
        output_filename = f"{file_name_only}.json"
        output_path = os.path.join(JSON_OUTPUT_DIR, output_filename)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(json_results, f, ensure_ascii=False, indent=2)
            
        print(f"  저장 완료: {output_filename} ({len(chunks)}개 청크)")

# 실행
if __name__ == "__main__":
    process_md_to_json()
    print("\n 모든 파일의 개별 JSON 변환이 완료되었습니다.")

🧠 임베딩 모델 로드 중...
📄 처리 중: 112신고.md
  ✅ 저장 완료: 112신고.json (8개 청크)
📄 처리 중: 가사근로자.md
  ✅ 저장 완료: 가사근로자.json (7개 청크)
📄 처리 중: 간선급버스운행.md
  ✅ 저장 완료: 간선급버스운행.json (25개 청크)
📄 처리 중: 간호법.md
  ✅ 저장 완료: 간호법.json (29개 청크)
📄 처리 중: 감사원징계.md
  ✅ 저장 완료: 감사원징계.json (15개 청크)
📄 처리 중: 개인정보호법.md
  ✅ 저장 완료: 개인정보호법.json (112개 청크)
📄 처리 중: 법원설치.md
  ✅ 저장 완료: 법원설치.json (2개 청크)
📄 처리 중: 소비세법.md
  ✅ 저장 완료: 소비세법.json (51개 청크)
📄 처리 중: 온실가스배출.md
  ✅ 저장 완료: 온실가스배출.json (11개 청크)

🚀 모든 파일의 개별 JSON 변환이 완료되었습니다.
